# <center>Analyse daily meassurements from HRV Logger App<center>
### <center>Databasis: feature computation from app<center>

In [1]:
import pandas as pd
from csv_prepare import csv_cp
from create_df import read_df
import seaborn as sns
import matplotlib.pyplot as plt
import seaborn.objects as so

# <center> User Input <center>

In [2]:
# select which measurements to show
select_measurement = ['ot_m', 'ot_a']   #so far just 2 sets are selectable
plt_select_length = 4                   #select at which character the measurements are divided for the plot

## Obsidian related plots ##
windowframe = '7D' # select how many days to show
select_rolling = 15 # select number of measurements for rolling median calculation

## <center> Setup Dataframes <center>

### Import data from filesystem

In [3]:
# copy files to workdir, create column measurement, format timestamp
csv_cp()

#create initial dataframe
df = read_df()

# add date column to df
df['date'] = df['time'].dt.date
df['date'] = pd.to_datetime(df['date'])

added file:  ot_m83_Features.csv


### Setup individual dataframes

In [ ]:
# dataframes per measurement
select_measurement_idx0 = df[df['measurement'].str.contains(select_measurement[0])]
select_measurement_idx0 = select_measurement_idx0.sort_values('measurement')
select_measurement_idx1 = df[df['measurement'].str.contains(select_measurement[1])]
select_measurement_idx1 = select_measurement_idx1.sort_values('measurement')

# grouped dataframes, used in calculate rolling values
gpDate_select_measurement_idx0 = select_measurement_idx0.groupby('date').median()
gpDate_select_measurement_idx0['rol_rmssd'] = gpDate_select_measurement_idx0['rmssd'].rolling(select_rolling).median()
gpDate_select_measurement_idx0['rol_hr'] = gpDate_select_measurement_idx0['hr'].rolling(select_rolling).median()

gpDate_select_measurement_idx1 = select_measurement_idx1.groupby('date').median()
gpDate_select_measurement_idx1['rol_rmssd'] = gpDate_select_measurement_idx1['rmssd'].rolling(select_rolling).median()
gpDate_select_measurement_idx1['rol_hr'] = gpDate_select_measurement_idx1['hr'].rolling(select_rolling).median()

# all measurements, long format, grouped by date, select column for plot
gp_MesDt = df.groupby(['measurement', 'date']).median()[['hr', 'rmssd']]
gp_MesDt.reset_index(drop= False, inplace= True)
gp_MesDt = gp_MesDt[gp_MesDt['measurement'].str.contains('|'.join(select_measurement))]
gp_MesDt['plt_select'] = gp_MesDt['measurement'].str[:plt_select_length]

##### used for plot

# create filter last 7 days
plt_df_1 = pd.DataFrame(gp_MesDt[gp_MesDt['measurement'].str.contains(select_measurement[0])])
plt_df_1.set_index('date', drop= True, inplace= True)
plt_df_1 = plt_df_1.last(windowframe)
plt_df_1.reset_index(drop= False, inplace= True)
plt_df_1 = pd.merge(plt_df_1,
                    gpDate_select_measurement_idx0[['rol_hr', 'rol_rmssd']],
                    on= 'date',
                    how= 'left')

plt_df_2 = pd.DataFrame(gp_MesDt[gp_MesDt['measurement'].str.contains(select_measurement[1])])
plt_df_2.set_index('date', drop= True, inplace= True)
plt_df_2 = plt_df_2.last(windowframe)
plt_df_2.reset_index(drop= False, inplace= True)
plt_df_2 = pd.merge(plt_df_2,
                    gpDate_select_measurement_idx1[['rol_hr', 'rol_rmssd']],
                    on= 'date',
                    how= 'left')

plt_df = pd.concat([plt_df_1, plt_df_2])
plt_df.reset_index(drop=True, inplace=True)

## check if all measurements are imported

In [ ]:
print('Morgenroutine: ', 
      select_measurement_idx0.sort_values('measurement')['measurement'].unique()[-1],
      '|', 
      len(select_measurement_idx0.sort_values('measurement')['measurement'].unique()),
      '|',
      select_measurement_idx0['time'].iloc[-1].day,'-',select_measurement_idx0['time'].iloc[-1].month,'-',select_measurement_idx0['time'].iloc[-1].year)

print('Abendroutine:  ', 
      select_measurement_idx1.sort_values('measurement')['measurement'].unique()[-1],
      '|', 
      len(select_measurement_idx1.sort_values('measurement')['measurement'].unique()),
      '|',
      select_measurement_idx1['time'].iloc[-1].day,'-',select_measurement_idx1['time'].iloc[-1].month,'-',select_measurement_idx1['time'].iloc[-1].year)

# <center> Print <center>

## set styling for all plots

In [ ]:
sns.set_theme(style= 'whitegrid',
              palette= 'tab10',
              context= 'notebook'
              )

## Overview plots

In [ ]:
select_measurement_idx1

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=1, figsize=(30,15) )

sns.lineplot(data=select_measurement_idx0,
             x='date',
             y='hr')
sns.lineplot(data=select_measurement_idx1,
             x='date',
             y='hr')
axes.legend(['hr_abend', 'variance', 'hr_morgen', 'variance'], fontsize=16)
axes.set_title("Heartrate", fontsize=16)

## Output for obsidian myHealth

In [ ]:
import matplotlib.dates as mdates

plt.close('all')
fig = plt.figure(figsize=(15,11))

# define grid and plot windows in main grid
sf1 = plt.subplot2grid((3,2), (0,0))    #links oben
sf2 = plt.subplot2grid((3,2), (0,1))    #rechts oben
sf3 = plt.subplot2grid((3,2), (1,0), colspan=2)
sf4 = plt.subplot2grid((3,2), (2,0), colspan=2)

# create plots with seaborn object interface
(
    so.Plot(plt_df,
            y='rmssd',
            x='date',
            color= 'plt_select')
    .add(so.Bars(alpha= 0.4), 
         so.Agg('median'), 
         so.Dodge(gap=.1))  # type: ignore
    .add(so.Line(linestyle= '--'), y= 'rol_rmssd')
    .label(title= 'rmssd',
           x= 'dates')
    .limit(y= (plt_df['rmssd'].min()-5, plt_df['rmssd'].max()+5))          
    .on(sf1)
    .plot()
)

(
    so.Plot(plt_df,
            y='hr',
            x='date',
            color= 'plt_select')
    .add(so.Bars(alpha= 0.4),
         so.Agg('median'), 
         so.Dodge(gap=.1))  # type: ignore
    .add(so.Line(linestyle= '--'), y= 'rol_hr')
    .label(title= 'hr',
           x= 'dates')
    .limit(y= (plt_df['hr'].min()-2, plt_df['hr'].max()+1))
    .on(sf2)
    .plot()
)

(
    so.Plot(select_measurement_idx0[select_measurement_idx0['date'] == max(select_measurement_idx0['date'])],
            y='rmssd',
            x='time',)
    .add(so.Bars(alpha= 0.4), 
         so.Agg('median'))
    .add(so.Line(linestyle= '--', linewidth= 1,), y= 'hr')
    .label(title= 'Morgenuebung',
           x= max(select_measurement_idx0['date']).date())
    .on(sf3)
    .plot()
)

(
    so.Plot(select_measurement_idx1[select_measurement_idx1['date'] == max(select_measurement_idx1['date'])],
            y='rmssd',
            x='time',)
    .add(so.Bars(alpha= 0.4, color= '#ff7f0e'), 
         so.Agg('median'))
    .add(so.Line(linestyle= '--', linewidth= 1, color= '#ff7f0e'), y= 'hr')
    .label(title= 'Abenduebung',
           x= max(select_measurement_idx1['date']).date())
    .on(sf4)
    .plot()
)
plt.tight_layout()